<a href="https://colab.research.google.com/github/TTTanya/Real_Or_Not_Kaggle/blob/main/RealOrNor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
%matplotlib inline 

In [ ]:
# Максимальное количество слов 
max_words = 10000
# Максимальная длина новости
maxlen = 20

In [ ]:
train = files.upload()

Saving dis_train.csv to dis_train.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(train['dis_train.csv']))

In [ ]:
X_train = df['text']
y_train = df['target']

In [ ]:
y_train

In [ ]:
token = Tokenizer(num_words=max_words)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen = maxlen)

In [ ]:
print(X_train[279])
print(sequences[279])

[ 878 1326  193 1240   13    4  914  137  716  259 1008    7   40    4
  479   44 1121  245   25 1649]
She's kinda hot played on the radio today. What's next? Disease to all? The apocalypse has started everyone. Be careful.
[878, 1326, 193, 1240, 13, 4, 914, 137, 716, 259, 1008, 7, 40, 4, 479, 44, 1121, 245, 25, 1649]


In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(max_words, 64, input_length=maxlen))
model_cnn.add(Conv1D(500, 7, padding='valid', activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])#0.795

In [ ]:
model_cnn_save_path = 'best_model_cnn.h5'
checkpoint_callback_cnn = ModelCheckpoint(model_cnn_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
model_ls = Sequential()
model_ls.add(Embedding(max_words, 64, input_length=maxlen))
model_ls.add(LSTM(16))
model_ls.add(Dense(1, activation='sigmoid'))
model_ls.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])#0.796

In [ ]:
model_lstm_save_path = 'best_model_lstm.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_lstm_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
model_gr = Sequential()
model_gr.add(Embedding(max_words, 32, input_length=maxlen))
model_gr.add(GRU(16))
model_gr.add(Dense(1, activation='sigmoid'))
model_gr.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])#0.794

In [ ]:
history1 = model_cnn.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1, callbacks=[checkpoint_callback_cnn])

Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 0.6811 - accuracy: 1.0000
Epoch 00001: val_accuracy improved from 0.80446 to 1.00000, saving model to best_model_cnn.h5
1/1 [==============================] - 0s 153ms/step - loss: 0.6811 - accuracy: 1.0000 - val_loss: 0.6399 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 0.6342 - accuracy: 1.0000
Epoch 00002: val_accuracy did not improve from 1.00000
1/1 [==============================] - 0s 17ms/step - loss: 0.6342 - accuracy: 1.0000 - val_loss: 0.5907 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - ETA: 0s - loss: 0.5821 - accuracy: 1.0000
Epoch 00003: val_accuracy did not improve from 1.00000
1/1 [==============================] - 0s 15ms/step - loss: 0.5821 - accuracy: 1.0000 - val_loss: 0.5395 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - ETA: 0s - loss: 0.5269 - accuracy: 1.0000
Epoch 00004: val_accuracy did not improve f

In [ ]:
history2 = model_ls.fit(X_train, y_train, epochs=15, batch_size=128, validation_split=0.1, callbacks=[checkpoint_callback_lstm])

In [ ]:
history3 = model_gr.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

In [ ]:
test = files.upload()

Saving dis_test.csv to dis_test.csv


In [ ]:
df_test = pd.read_csv(io.BytesIO(test['dis_test.csv']))

In [ ]:
test_sequences = token.texts_to_sequences(df_test['text'])
X_test = pad_sequences(test_sequences, maxlen = maxlen)

In [ ]:
model_cnn.load_weights(model_cnn_save_path)
pred_cnn = model_cnn.predict_classes(X_test)

In [ ]:
model_ls.load_weights(model_lstm_save_path)
pred_ls = model_ls.predict_classes(X_test)

In [ ]:
pred_gr = model_gr.predict_classes(X_test)

In [ ]:
pred_cnn[100][0]

0

In [ ]:
arr = []
j=0
for i in pred_gr:
  print(i[0])
  arr.append(i[0])

In [ ]:
arr

In [ ]:
model_df = pd.DataFrame({'id': df_test['id'], 'target': [i[0] for i in pred_cnn]})
model_df.to_csv('disaster.csv', index = False)

In [ ]:
model_df

In [ ]:
files.download('disaster.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>